# Disturbance & Event Guides: TimeSeriesDisturbance

This notebook provides a guide to the `TimeSeriesDisturbance` model. This component is essential for simulating external factors that change over time, such as fluctuating water demand from a city, or a known inflow pattern from an upstream source.

## 1. How It Works

The `TimeSeriesDisturbance` model takes a predefined series of timestamps and corresponding values. At any point during the simulation, it uses **linear interpolation** to calculate the correct output value for the current simulation time `t`. This allows for smooth, continuous signals even if your data points are far apart.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.disturbances.timeseries_disturbance import TimeSeriesDisturbance
from water_system_sdk.src.chs_sdk.modules.modeling.storage_models import LinearTank

## 2. Code Example: Simulating Variable Water Demand

We will simulate a `LinearTank` representing a water tower. The outflow from this tower will be determined by a `TimeSeriesDisturbance` that represents the water demand of a small town over a 24-hour period (high demand in the morning and evening, low at night).

In [ ]:
# 1. Define the time series for water demand (m^3/s)
# Times are in seconds from the start of the day
demand_times = [0, 6*3600, 8*3600, 12*3600, 18*3600, 20*3600, 24*3600]
demand_values = [0.5, 0.6,    1.5,      1.0,      1.8,      1.2,      0.5]

# 2. Initialize the models
demand_model = TimeSeriesDisturbance(times=demand_times, values=demand_values)
water_tower = LinearTank(area=2000.0, initial_level=10.0)

# 3. Simulation Setup
dt = 600 # 10-minute time step
n_steps = int(24 * 3600 / dt)
history = {"time": [], "level": [], "demand": []}

# 4. Simulation Loop
for i in range(n_steps):
    current_time = i * dt
    
    # Get the current demand from the disturbance model
    demand_model.step(dt=dt, t=current_time)
    current_demand = demand_model.output
    
    # Set the tank's inputs
    water_tower.input.inflow = 1.0 # Constant inflow from a pump
    water_tower.input.demand_outflow = current_demand
    
    # Run the tank model
    water_tower.step(dt=dt)
    
    # Record results
    history["time"].append(current_time / 3600) # time in hours
    history["level"].append(water_tower.level)
    history["demand"].append(current_demand)

print("Simulation complete.")

## 3. Visualization

Plotting the tank's water level against the water demand shows how the external disturbance affects the system state.

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot Level
color = 'tab:blue'
ax1.set_xlabel('Time (hours)')
ax1.set_ylabel('Water Tower Level (m)', color=color)
ax1.plot(history['time'], history['level'], color=color, label='Water Level')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

# Plot demand on a second y-axis
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Water Demand (m³/s)', color=color)
ax2.plot(history['time'], history['demand'], color=color, linestyle='--', label='Demand')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Water Tower Level with Variable Demand')
fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes)
plt.show()

The plot shows the water level in the tower decreasing during the morning and evening peak demand periods and recovering during the low-demand periods of midday and overnight. This demonstrates how the `TimeSeriesDisturbance` model can be used to create realistic, time-varying scenarios for testing and analysis.